In [1]:
!pip install six google-api-python-client>=1.6.7 kaggle>=1.3.9 numpy>=1.20 oauth2client pandas>=0.22.0 psutil>=5.4.3 py-cpuinfo>=3.3.0 scipy>=0.19.1 tensorflow-hub>=0.6.0 tensorflow-model-optimization>=0.4.1 tensorflow-datasets gin-config tf_slim>=1.1.0 Cython matplotlib pyyaml>=5.1,<5.4.0 opencv-python-headless Pillow pycocotools seqeval sentencepiece sacrebleu immutabledict

The system cannot find the file specified.


In [2]:
import sys
sys.path.append('models/research')
sys.path.append('models/research/slim')  # Also add the slim directory to the path

In [4]:
!protoc object_detection/protos/*.proto --python_out=.

Invalid file name pattern or missing input file "object_detection/protos/*.proto"


In [3]:
import os
import numpy as np
import tensorflow as tf
import cv2
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Path to the pre-trained model checkpoint and the label map file
MODEL_NAME = 'ssd_mobilenet_v2_coco'
PATH_TO_CKPT = 'newmodel/frozen_inference_graph.pb'
PATH_TO_LABELS = 'newmodel/mscoco_label_map.pbtxt'
NUM_CLASSES = 90  # Number of classes in the COCO dataset

# Load the pre-trained model
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

# Load the label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

# Path to the test image
PATH_TO_TEST_IMAGE = '3.jpg'

# Load the test image
image = cv2.imread(PATH_TO_TEST_IMAGE)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_expanded = np.expand_dims(image_rgb, axis=0)

# Run object detection
with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:
        # Get handles to input and output tensors
        ops = tf.compat.v1.get_default_graph().get_operations()
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict = {}
        for key in ['num_detections', 'detection_boxes', 'detection_scores', 'detection_classes', 'detection_masks']:
            tensor_name = key + ':0'
            if tensor_name in all_tensor_names:
                tensor_dict[key] = tf.compat.v1.get_default_graph().get_tensor_by_name(tensor_name)

        # Run inference
        output_dict = sess.run(tensor_dict, feed_dict={ 'image_tensor:0': image_expanded })

        # All outputs are float32 numpy arrays, so convert types as needed
        output_dict['num_detections'] = int(output_dict['num_detections'][0])
        output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.int64)
        output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
        output_dict['detection_scores'] = output_dict['detection_scores'][0]

# Visualization of the results of detection
vis_util.visualize_boxes_and_labels_on_image_array(
    image_rgb,
    output_dict['detection_boxes'],
    output_dict['detection_classes'],
    output_dict['detection_scores'],
    category_index,
    use_normalized_coordinates=True,
    line_thickness=8)

# Display the output image with detected objects
cv2.imshow('Object Detection', cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()


NotFoundError: Op type not registered 'TFLite_Detection_PostProcess' in binary running on 3520-QGUAN2. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.